Prepare the execution environment

In [ ]:
import osmnx as ox
from helper_funcs.gridding_helpers import Grid_Generation_Task_Manager, generate_grid, generate_file_name
from helper_funcs.setting_helpers import load_yaml_config_file

Now load the settings from the settings yaml file for the area (lake) of interest:

In [ ]:
settings_yaml_filepath = './settings/settings_talsperre_malter.yaml'
settings = load_yaml_config_file(settings_yaml_filepath)

tile_edge_lengths = settings['sensor_line_length_meter']
area_name = settings['area_name']
polygon_threshold = settings['polygon_threshold']

### How To find the area of interest?

First you need to find the area of interest...
Let's use the OSMnx module to get data through the OSM Overpass API.

Either use the OSM ID to find the geometry:

In [ ]:
malter_area = ox.geocode_to_gdf(query=["R2141671"], by_osmid=True)  # river dam Malter in Saxony, Germany

malter_area_exploded = malter_area.geometry.explode(index_parts=True)  # no index_parts / .tolist()
biggest_area = max(malter_area_exploded, key=lambda a: a.area)

or build a search query and feed it to the OSMnx place search:

In [ ]:
# place = "Talsperre Malter, Saxony, Germany"
# tags = {'water': True}
# ox_test = ox.geometries_from_place(place, tags)
# ox_test.head()

### Now draw the regions inside the area of interest

Create areas to examine as Polygons for every path (scanner) line width with a Task Manager.
This uses ipyleaflet to draw interactive Maps.

If the saved areas overlap between smaller and greater tile widths: The greater width gets prioritized.

Create a Task Manager with the region of interest as area specification (must be MultiPolygon):

In [ ]:
task_manager = Grid_Generation_Task_Manager(tile_edge_lengths, polygon_threshold, biggest_area, area_name)

Wanna set a specific region for one tile (scanner) line width?
Use the task manager set_areas() methode and use the Polygon Draw functionality.
If you don't draw anything specific for one scanner line width defined in the settings there will be no restrictions and the grid generation tries to fill the whole area of interest.

In [ ]:
task_manager.set_areas()

If you are done with all your region specifications you can use the generate_grid function and start the grid generation process by giving it the task manager. It calls the task manager internal methode extract_tasks to get all the necessary data for the grid generation.

In [ ]:
grid_gdf = generate_grid(task_manager)

if not grid_gdf.empty:
    # save best results
    file_name = generate_file_name(settings['area_name'])
    grid_gdf.to_file(filename=f'./geodataframes/{file_name}_grid.geojson', driver="GeoJSON")
    print("Successfully finished grid generation and saved geometry to file!\n",
          f'./geodataframes/{file_name}_grid.geojson')


In the end you can examine your results from the grid generation process by using the GeoDataframe explore method (which draws a folium Map).

In [ ]:
grid_gdf.explore('covered_area', cmap='Spectral')